In [2]:
from colmap_testing.colmap_helpers import read_write_model
import os

In [18]:
path_to_model = os.path.join("colmap_testing", "colmap_ws", "sparse","0")

In [20]:
colmap_model = read_write_model.read_model(path_to_model, ".bin")

({1: Camera(id=1, model='SIMPLE_RADIAL', width=1088, height=1080, params=array([8.39206178e+02, 5.44000000e+02, 5.40000000e+02, 4.75940597e-02]))},
 {1: Image(id=1, qvec=array([ 0.96182572,  0.05411164, -0.24629483, -0.1063112 ]), tvec=array([ 3.30278924,  0.51305215, -6.08837689]), camera_id=1, name='145.png', xys=array([[412.11367798, 756.51055908],
         [638.42089844, 554.32855225],
         [157.23048401, 488.11825562],
         ...,
         [178.32836914, 728.24902344],
         [435.21884155, 927.30102539],
         [294.79214478, 819.7947998 ]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1])),
  2: Image(id=2, qvec=array([ 0.98860302, -0.02449892, -0.12292113, -0.08339226]), tvec=array([ 0.95896991,  0.43794351, -3.17217346]), camera_id=1, name='3.png', xys=array([[854.52264404, 673.43426514],
         [405.73410034, 611.51501465],
         [264.58013916, 535.20916748],
         ...,
         [256.44671631, 562.75299072],
         [590.67895508, 443.30993652],
         [